In [ ]:
# https://openrouter.ai/settings/keys

In [1]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.2 MB/s eta 0:00:00


In [2]:
!pip install langchain-experimental

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 5.5 MB/s eta 0:00:00


In [3]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 66.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 77.3 MB/s eta 0:00:00


In [4]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 25.7 MB/s eta 0:00:00


In [5]:
!git clone https://github.com/sarveswarraj/IBM.git

Cloning into 'IBM'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (9/9), 2.52 MiB | 4.74 MiB/s, done.


In [6]:
import os
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
from google.colab import userdata

# Step 1: Set your OpenRouter API key and endpoint
os.environ["OPENAI_API_KEY"] = userdata.get("OPENROUTER_OPENAPI") # Replace with your actual OpenRouter API key
os.environ["OPENAI_API_BASE"] = "https://openrouter.ai/api/v1"

# Step 2: Initialize the LLM
llm = ChatOpenAI(
    model="mistralai/mistral-7b-instruct",  # or try "meta-llama/llama-3-8b-instruct", etc.
    temperature=0.7,
    openai_api_base=os.environ["OPENAI_API_BASE"],
    openai_api_key=os.environ["OPENAI_API_KEY"],
    request_timeout=60,
    verbose=True
)

# Step 3: Interact with the model (normal chat mode)
while True:
    query = input("\nYou: ")
    if query.lower() in ["exit", "quit", "bye"]:
        print("Goodbye!")
        break

    response = llm([HumanMessage(content=query)])
    print("\nAssistant:", response.content)


/tmp/ipython-input-202412850.py:11: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(



You: hi


/tmp/ipython-input-202412850.py:27: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm([HumanMessage(content=query)])



Assistant:  Hello! How can I help you today? Is there something specific you would like to talk about or ask? I'm here to assist with a wide range of topics, from answering questions to providing information and recommendations. Let me know what you need!


KeyboardInterrupt: Interrupted by user

In [10]:
import os
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA # RAG - To retrieve the contents from the pdfs or any docs
from langchain.chains.llm import LLMChain  # It's LangChain function to wrap the tools/agents to create a seemless workflow
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.prompts import PromptTemplate  # To provide the equipped prompt
from langchain_community.document_loaders import PDFPlumberLoader # To read the pdf
from langchain_experimental.text_splitter import SemanticChunker # Text chunks
from langchain_community.embeddings import HuggingFaceEmbeddings # Embed the texts or contents
from langchain_community.vectorstores import FAISS  # Vector DB
from google.colab import userdata

# Set your OpenRouter API key and endpoint
os.environ["OPENAI_API_KEY"] = userdata.get("OPENROUTER_OPENAPI")
os.environ["OPENAI_API_BASE"] = "https://openrouter.ai/api/v1"  # Endpoint of the openRouter

# Load and process PDF
loader = PDFPlumberLoader("/content/IBM/INDIAN CONSTITUTION PDF.pdf")
docs = loader.load()
print("Pages loaded:", len(docs))



Pages loaded: 402


In [16]:
# PROCESS 1: CHUNK process
# embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Semantic splitter with custom
# With SemanticChunker (in LangChain), you don't define chunk_size directly like in other splitters
# (CharacterTextSplitter, RecursiveCharacterTextSplitter, etc.).
# Instead, semantic chunking decides where to split based on semantic similarity between segments, not on fixed sizes.
splitter = SemanticChunker(
    embeddings=HuggingFaceEmbeddings(),
    breakpoint_threshold_type="percentile", # or standard_deviation
    breakpoint_threshold_amount=90, # Value for the threshold (e.g. 95 = 95th percentile distance between embeddings)
    buffer_size=1 # (Optional) Adds context from neighboring chunks
)

/tmp/ipython-input-1277045419.py:9: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings=HuggingFaceEmbeddings(),


In [15]:
# Split document
chunks = splitter.split_documents(docs)
for chunk in chunks:
    print(chunk.page_content)

THE CONSTITUTION OF INDIA
[As on 26th November, 2021]

PREFACE
This is the fifth pocket size edition of the Constitution of India in the
diglot form. In this edition, the text of the Constitution of India has been
brought up-to-date by incorporating therein all the amendments up to the
Constitution (One Hundred and Fifth Amendment) Act, 2021. The foot notes
below the text indicate the Constitution Amendment Acts by which such
amendments have been made. The Constitution (One Hundredth Amendments) Act, 2015 containing
details of acquired and transferred territories between the Governments of
India and Bangladesh has been provided in APPENDIX I. The Constitution (Application to Jammu and Kashmir) Order, 2019 and
the declaration under article 370(3) of the Constitution have been provided
respectively in Appendix II and Appendix III for reference. New Delhi; Dr.
Reeta Vasishta,
Secretary to the Government of India. 2

THE CONSTITUTION OF INDIA
____________
CONTENTS
__________
PREAMBLE
PART 

In [18]:
# PROCESS 2: Vector DB
embedder = HuggingFaceEmbeddings()
vector = FAISS.from_documents(chunks, embedder)
retriever = vector.as_retriever(search_type="similarity", search_kwargs={"k": 2})

'''
# Try with some queries for similarity search
results = retriever.get_relevant_documents("What is the remedy for cold and flu?")

for doc in results:
    print(doc.page_content)
'''

/tmp/ipython-input-1099814036.py:2: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embedder = HuggingFaceEmbeddings()


'\n# Try with some queries for similarity search\nresults = retriever.get_relevant_documents("What is the remedy for cold and flu?")\n\nfor doc in results:\n    print(doc.page_content)\n'

In [19]:
# PROCESS 3: LLM MODEL CHOOSING
# Use ChatOpenAI with OpenRouter backend
llm = ChatOpenAI(
    model="meta-llama/llama-3-8b-instruct",   #"mistralai/mistral-7b-instruct"
    temperature=1.3, # It's a less temp -> so it'll give you some relative information about the content.
    openai_api_base=os.environ["OPENAI_API_BASE"],
    openai_api_key=os.environ["OPENAI_API_KEY"],
    request_timeout=60,
)

In [20]:
# PROCESS 4: PROMPT CREATION
prompt = """
You are a helpful assistant.
Use the following pieces of context to answer the question at the end.
Answer only using the context and be concise (3–4 sentences).

Context: {context}

Question: {question}

Answer:
"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(prompt)

In [21]:
# PROCESS 5: LANGCHAIN CREATION - LLM & PROMPT
llm_chain = LLMChain(llm=llm, prompt=QA_CHAIN_PROMPT, verbose=True)

document_prompt = PromptTemplate(
    input_variables=["page_content", "source"],
    template="Context:\ncontent:{page_content}\nsource:{source}",
)

# DOCUMENT CHAIN PROCESS
combine_documents_chain = StuffDocumentsChain(
    llm_chain=llm_chain,
    document_variable_name="context",
    document_prompt=document_prompt,
    callbacks=None,
)

/tmp/ipython-input-1143679950.py:2: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(llm=llm, prompt=QA_CHAIN_PROMPT, verbose=True)
/tmp/ipython-input-1143679950.py:10: LangChainDeprecationWarning: This class is deprecated. Use the `create_stuff_documents_chain` constructor instead. See migration guide here: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain/
  combine_documents_chain = StuffDocumentsChain(


In [22]:
# PROCESS 6: WRAPPING ALL TOGETHER ALONG WITH RETRIEVER
qa = RetrievalQA(
    combine_documents_chain=combine_documents_chain,
    retriever=retriever,
    return_source_documents=True,
    verbose=True,
)

/tmp/ipython-input-765820220.py:2: LangChainDeprecationWarning: This class is deprecated. Use the `create_retrieval_chain` constructor instead. See migration guide here: https://python.langchain.com/docs/versions/migrating_chains/retrieval_qa/
  qa = RetrievalQA(


In [28]:
# Example query
result = qa("list all the articles")
print("Answer:", result["result"])



> Entering new RetrievalQA chain...


> Entering new LLMChain chain...
Prompt after formatting:

You are a helpful assistant.
Use the following pieces of context to answer the question at the end.
Answer only using the context and be concise (3–4 sentences).

Context: Context:
content:Subs. by the Constitution (One Hundred and First Amendment) Act, 2016, s. 15, for the
words and figures "article 162 or article 241" (w.e.f. 16-9-2016).
source:/content/IBM/INDIAN CONSTITUTION PDF.pdf

Context:
content:62. The institutions known at the commencement of this Constitution as
the National Library, the Indian Museum, the Imperial War Museum, the
Victoria Memorial and the Indian War Memorial, and any other like institution
financed by the Government of India wholly or in part and declared by
Parliament by law to be an institution of national importance. 63. The institutions known at the commencement of this Constitution as the
Benares Hindu University, the Aligarh Muslim University and the 1[